### 1. Предобработка заголовков, извлечение текста страниц из html-кода

In [3]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
stop_tokens = []

def is_word(token):
    legal_chars = '0123456789abcdefghijklmnopqrstuvwxyzабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    for c in legal_chars:
        if c in token:
            return True
    stop_tokens.append(token)
    return False

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token.strip() for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation
              and is_word(token)]
    
    text = " ".join(tokens)
    
    return text

In [5]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = preprocess_text(title)

# stop_tokens

In [ ]:
import os
from html.parser import HTMLParser

contents = ''

class Parser(HTMLParser):
    def handle_data(self, data):
        global contents
        tag = self.get_starttag_text()
        if tag is not None and tag.find('script') == tag.find('style') == -1:
            contents += data.strip() + ' '

directory = !pwd
directory = directory[0]
os.makedirs(directory + '/content/preprocessed')

parser = Parser()

for dirname, _, filenames in os.walk(directory + '/content'):
    if dirname.find('preprocessed') != -1: #начнет залезать в созданную папку preprocessed, надо пресечь
        break
    for filename in filenames:
        file = os.path.join(dirname,filename)
        with open(file, 'r') as f:
            strings = f.readlines()
            strings = [s.strip() for s in strings[1:] ] #убираем адрес сайта, находящийся на 1 строке
            text = '\n'.join(strings)
            parser.feed(text)
            
            newfile = file[:file.rfind('/')] + '/preprocessed/' + filename[:-4] + '_preprocessed.dat'
            
            with open(newfile, 'w') as f1:
                print(contents, file = f1)
            print(len(contents), 'chars in', newfile)
            parser.reset()
            contents = ''

In [8]:
df = pd.read_csv('train_groups.csv')
df['title'] = pd.Series(dtype='object')
for i, row in df.iterrows():
    df['title'][i] = doc_to_title[row['doc_id']]
df.to_csv('train_preprocessed.csv', index=False)
df

/home/danroor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,pair_id,group_id,doc_id,target,title
0,1,1,15731,0,ваз 21213 замена подшипник ступица нива
1,2,1,14829,0,ваз 2107 опт сочи сравнивать цена купить потре...
2,3,1,15764,0,купить ступица лад калина2 трансмиссия переход...
3,4,1,17669,0,классика 21010 21074
4,5,1,14852,0,ступица нива замена подшипник свой рука
...,...,...,...,...,...
11685,11686,129,26672,0,апреленок 2014 6
11686,11687,129,25838,0,gästebuch
11687,11688,129,25703,0,jizolofej archive
11688,11689,129,27885,0,звать парень диана шурыгина пусть говорить диа...


In [9]:
df = pd.read_csv('test_groups.csv')
df['title'] = pd.Series(dtype='object')
for i, row in df.iterrows():
    df['title'][i] = doc_to_title[row['doc_id']]
df.to_csv('test_preprocessed.csv', index=False)
df

/home/danroor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,pair_id,group_id,doc_id,title
0,11691,130,6710,прописывать админка кс 1.6 друг youtube
1,11692,130,4030,скачать sgl rp доработка слива мода mysql rp r...
2,11693,130,5561,прописывать админка кс 1.6 counter strike ката...
3,11694,130,4055,прописывать простой админка кс 1 6
4,11695,130,4247,подбор админ сервер код 4 архив форум ozone
...,...,...,...,...
16622,28313,309,16637,ответ mail ru полезно кушать творог утро худет...
16623,28314,309,16759,творог полезный свойство лечение творог женски...
16624,28315,309,15358,творог полезный опасный свойство творог
16625,28316,309,17287,ответ mail ru полезный творог
